# 数据准备

## 数据导入

In [1]:
from ngboost import NGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pandas as pd
data = pd.read_csv('/Users/apple/Documents/ML_Project/ML - 2.1/Data/国际西班牙数据.csv')
display(data.describe())

,wind_speed,wind_direction,wind_power
count,18087.000000,18087.000000,17988.000000
mean,5.970865,173.388069,2955.932230
std,2.990040,96.471277,3555.648744
min,0.350000,0.000000,0.000000
25%,3.960000,91.000000,83.712500
50%,5.450000,167.000000,1515.720000
75%,7.440000,256.000000,4588.337500
max,27.130000,359.000000,16163.410000


In [2]:
data['Date'] = data['Date'].apply(lambda data: data.split('/'))
data['Year'] = data['Date'].apply(lambda data: int(data[2].split(' ')[0]))
data['Month'] = data['Date'].apply(lambda data: int(data[1]))
data['Day'] = data['Date'].apply(lambda data: int(data[0]))
data['Hour'] = data['Date'].apply(lambda data: int(data[2].split(' ')[1].split(':')[0]))
data.drop('Date', inplace=True, axis=1)

In [3]:
X = data[['wind_direction','wind_speed']]
Y = data['wind_power']
X_train = X.iloc[6426:10427]
Y_train = Y.iloc[6426:10427]
X_test = X.iloc[14389:17872]
Y_test = Y.iloc[14389:17872]

# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=520)

from sklearn.preprocessing import MinMaxScaler
X_Scaler = MinMaxScaler()
X_train = X_Scaler.fit_transform(X_train)
X_test = X_Scaler.transform(X_test)

Y_Scaler = MinMaxScaler()
Y_train = Y_Scaler.fit_transform(Y_train.values.reshape(-1,1)).reshape(len(Y_train),)
Y_test = Y_Scaler.transform(Y_test.values.reshape(-1,1)).reshape(len(Y_test),)

# 数据分析

In [4]:
data['ws_sin(wd)'] = data['wind_speed']*np.sin(data['wind_direction'])
data['ws_cos(wd)'] = data['wind_speed']*np.cos(data['wind_direction'])

from sklearn.preprocessing import MinMaxScaler
data['ws_wd'] = data['wind_speed']*data['wind_direction']
data['ws(Scaled)_wd(Scaled)'] = \
MinMaxScaler().fit_transform(data['wind_speed'].values.reshape(-1,1))*\
MinMaxScaler().fit_transform(data['wind_direction'].values.reshape(-1,1))

data['wind_power(Scaled)'] = \
MinMaxScaler().fit_transform(data['wind_power'].values.reshape(-1,1))

df = data[6426:10427]

## 可视化


In [239]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
from sklearn.preprocessing import MinMaxScaler

def plot_module(month, day, figsize=(14,16), save_fig=False):
    fig, axes = plt.subplots(4,2,figsize=figsize)
    fig.suptitle('Year\Month\Day: 2017\{}\{}'.format(month,day), fontsize=15)
    df[(df['Month']==month) & (df['Day']==day)]['wind_speed'].plot(ax=axes[0,0],title='wind_speed', ylim=[0,7])
    df[(df['Month']==month) & (df['Day']==day)]['wind_direction'].plot(ax=axes[1,0],title='wind_direction', ylim=[90,360])
    df[(df['Month']==month) & (df['Day']==day)]['wind_power'].plot(ax=axes[2,0], title='wind_power', ylim=[0,5000])
    df[(df['Month']==month) & (df['Day']==day)]['ws_sin(wd)'].plot(ax=axes[0,1],title='wind_speed*sin(wind_direction)')
    df[(df['Month']==month) & (df['Day']==day)]['ws_cos(wd)'].plot(ax=axes[1,1],title='wind_speed * cos(wind_direction)')
    df[(df['Month']==month) & (df['Day']==day)]['ws_wd'].plot(ax=axes[2,1],title='wind_speed*wind_direction')
    # df[(df['Month']==month) & (df['Day']==day)]['ws(Scaled)_wd(Scaled)'].plot(ax=axes[3,1],title='wind_speed*wind_direction(Both Scaled)')


    pd.Series(MinMaxScaler().fit_transform\
    (df[(df['Month']==month) & (df['Day']==day)]\
     ['wind_speed'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[3,0])

    pd.Series(MinMaxScaler().fit_transform\
    (df[(df['Month']==month) & (df['Day']==day)]\
     ['wind_direction'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[3,0],title='wind_speed & wind_direction')
    if save_fig:
        plt.savefig('/Users/apple/Documents/ML_Project/ML - 2.1/figure/2017\{}\{}.png'.format(month,day))
    plt.close()

In [240]:
from tqdm.notebook import tqdm as tqdm
for day in tqdm(np.arange(1,11)):
    plot_module(month=10, day=day, figsize=(14,16), save_fig=True)

In [268]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
from sklearn.preprocessing import MinMaxScaler

def plot_module2(month, day, figsize=(14,14), save_fig=False, close_fig=True):
    fig, axes = plt.subplots(3,2,figsize=figsize)
    fig.suptitle('Year\Month\Day: 2017\{}\{}'.format(month,day), fontsize=15)
    df[(df['Month']==month) & (df['Day']==day)]['wind_speed'].plot(ax=axes[0,0],title='wind_speed', ylim=[0,7])
    df[(df['Month']==month) & (df['Day']==day)]['wind_direction'].plot(ax=axes[1,0],title='wind_direction', ylim=[90,360])
    df[(df['Month']==month) & (df['Day']==day)]['wind_power'].plot(ax=axes[2,0], title='wind_power', ylim=[0,5000])
    df[(df['Month']==month) & (df['Day']==day)]['wind_power(Scaled)'].plot(ax=axes[1,1],title='wind_power(Scaled)')
    df[(df['Month']==month) & (df['Day']==day)]['ws_wd'].plot(ax=axes[2,1],title='wind_speed * wind_direction')
    
    pd.Series(MinMaxScaler().fit_transform\
              (df[(df['Month']==month) & (df['Day']==day)]\
               ['wind_speed'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[0,1])

    pd.Series(MinMaxScaler().fit_transform\
              (df[(df['Month']==month) & (df['Day']==day)]\
               ['wind_direction'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[0,1],title='wind_speed(Scaled) & wind_direction(Scaled)')
    if save_fig:
        plt.savefig('/Users/apple/Documents/ML_Project/ML - 2.1/figure2/2017\{}\{}.png'.format(month,day))
    if close_fig:
        plt.close()

In [269]:
from tqdm.notebook import tqdm as tqdm
for day in tqdm(np.arange(1,11)):
    plot_module2(month=10, day=day, figsize=(14,16), save_fig=True, close_fig=True)

In [10]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
from sklearn.preprocessing import MinMaxScaler

def plot_module3(year,month, day, data, figsize=(14,14), save_fig=False, close_fig=True):
    df = data
    fig, axes = plt.subplots(3,2,figsize=figsize)
    fig.suptitle('Year\Month\Day: {}\{}\{}'.format(year,month,day), fontsize=15)
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_speed'].plot(ax=axes[0,0],title='wind_speed', ylim=[0,7])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_direction'].plot(ax=axes[1,0],title='wind_direction', ylim=[90,360])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_power'].plot(ax=axes[2,0], title='wind_power', ylim=[0,5000])
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['wind_power(Scaled)'].plot(ax=axes[1,1],title='wind_power(Scaled)')
    df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]['ws_wd'].plot(ax=axes[2,1],title='wind_speed * wind_direction')
    
    pd.Series(MinMaxScaler().fit_transform\
              (df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]\
               ['wind_speed'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[0,1])

    pd.Series(MinMaxScaler().fit_transform\
              (df[(df['Year']==year) & (df['Month']==month) & (df['Day']==day)]\
               ['wind_direction'].values.reshape(-1,1)).reshape(24,)).plot(ax=axes[0,1],title='wind_speed(Scaled) & wind_direction(Scaled)')
    if save_fig:
        plt.savefig('/Users/apple/Documents/ML_Project/ML - 2.1/figure3/{}\{}\{}.png'.format(year,month,day))
    if close_fig:
        plt.close()

In [11]:
from tqdm.notebook import tqdm as tqdm
for month in tqdm(np.arange(1,13)):
    plot_module3(year=2018, month=month, day=1, data=data, figsize=(14,16), save_fig=True, close_fig=True)